In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mne.filter import filter_data
from scipy.signal import find_peaks_cwt
from entropy import shannon_entropy, sample_entropy
import time

In [154]:
trialData = np.load('C:/data/processed/ESM_pilot/trials.npy')
esm = pd.read_csv('C:\data\processed\ESM_pilot\esm.csv')

In [155]:
alignedFeatures = extractFeatures(trialData,esm,100,windowLength=60)
alignedFeatures.to_csv('C:/data/processed/ESM_pilot/features.csv',index=False)

C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


1.1299769878387451
1.1030516624450684
1.2885560989379883
1.2775838375091553
1.2167482376098633
1.1399521827697754
1.110032320022583
1.1060447692871094
1.1279828548431396
1.1020536422729492
1.1579039096832275
1.162893533706665
1.1469323635101318
1.1977977752685547
1.362358808517456
1.4959995746612549
1.3653504848480225
1.1997919082641602
1.2466669082641602
1.181840419769287
1.1648859977722168
1.2007899284362793
1.3164806365966797
1.1738612651824951
1.0920815467834473
1.1229970455169678
1.1220004558563232
1.1798465251922607
1.2366924285888672
1.1309776306152344
1.2546451091766357
1.1439440250396729
1.2785801887512207
1.1220014095306396
1.144939661026001
1.3124892711639404
1.19380784034729
1.128011703491211
1.1040215492248535
1.1020557880401611
1.1339662075042725
1.1110293865203857
1.1279850006103516
1.0970721244812012
1.1399517059326172
1.1439423561096191
1.148927927017212
1.3793127536773682
1.1200060844421387
1.1868278980255127
1.1678695678710938
1.2995266914367676
1.2087986469268799
1.

In [149]:
def extractFeatures(data, esm,sr, windowLength=60):

    numSamples=data.shape[0]
    # Getting number and names of features
    tremorNames, _ = tremorFeatures(data[0,:windowLength*sr,:], sr,windowLength=windowLength)
    bradyNames, _ = bradykinesiaFeatures(data[0,:windowLength*sr,:], sr,windowLength=windowLength)
    cols=[]
    for s in ['L','R','C']:
        cols.extend([c + s for c in tremorNames])
        cols.extend([c + s for c in bradyNames])
    cols.extend(esm.keys())
    aligned = pd.DataFrame(columns=cols)
    accelerometerChannel = [a + b for a in  [0,6,12] for b in range(3)]
    for beep in range(data.shape[0]):
        t=time.time()
        allFeat = []
        numWindows = int(data.shape[1]/sr/windowLength)
        buff = data[beep,:,:]
        buff[:,accelerometerChannel] = (buff[:,accelerometerChannel].T - np.mean(buff[:,accelerometerChannel].T,axis=0)).T
        buff[:,accelerometerChannel] = filter_data(buff[:,accelerometerChannel].T,sr,0,3,method='iir',verbose='WARNING').T
        for s,sID in enumerate([range(6),range(6,12),range(12,18)]):
            
            features=np.zeros((numWindows,len(tremorNames)+len(bradyNames)))
            for i in range(0,numWindows):
                win = i * windowLength * sr
                _, features[i,:len(tremorNames)] = tremorFeatures(buff[win:win+windowLength*sr,sID],sr,windowLength=windowLength)
                _, features[i,len(tremorNames):] = bradykinesiaFeatures(buff[win:win+windowLength*sr,sID],sr,windowLength=windowLength)
            allFeat.append(features)
        allFeat = np.concatenate(allFeat,axis=1)
        allFeat = np.concatenate((allFeat, np.matlib.repmat(esm.iloc[beep,:],numWindows,1)),axis=1)
        aligned = aligned.append(pd.DataFrame(allFeat,columns = cols),ignore_index=True)
        print(time.time()-t)
        # Add the power between 3.6 and 9.4
        #Timestamp at beginning of each window
        #alignedTimes.append(startTime + pd.Timedelta('%d s ' % (i * windowLength)))
    return aligned

In [146]:
def tremorFeatures(windowData,sr,windowLength=60):
    gyroChannel={'X':3,'Y':4,'Z':5} # gyro is xyz 3-4-5
    if windowData.shape[0]!=sr*windowLength:
        print(windowData.shape,sr*windowLength)
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    selected=np.logical_and(freq>3.5,freq<7.5)
    features=[]
    featureNames=[]
    for ch in gyroChannel.keys():
        spec = np.log(np.sum(np.abs(np.fft.rfft(windowData[:,gyroChannel[ch]]))[selected]))
        features.append(spec)
        featureNames.append('BandPower' + ch)
    return featureNames, features

In [145]:
def bradykinesiaFeatures(windowData,sr,windowLength=60):
    features=[]
    featureNames=[]
    accelerometerChannel={'X':0,'Y':1,'Z':2} # assuming acc xyz 0-1-2 is
    
    

    # lowpass filter signal <3 Hz
    
    # Features: (Patel et al IEEE 2009)
    # rms
    # range
    # entropy
    # normalized cross-correlation value and time point
    # dominat frequency and ratio between dominant and rest
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    
    for ch in accelerometerChannel.keys():
        #ent = shannon_entropy(windowData[:,accelerometerChannel[ch]])
        #features.append(ent)
        #featureNames.append('Entropy' + ch)
        
        spec = np.abs(np.fft.rfft(windowData[:,accelerometerChannel[ch]]))
        domFreq = freq[np.argmax(spec)]
        features.append(domFreq)
        featureNames.append('DomFreq' + ch)
        
        domEnergyRatio = np.max(spec) / np.sum(spec)
        features.append(domEnergyRatio)
        featureNames.append('DomEnergyRatio' + ch)
        
        rms = np.sqrt(np.mean(windowData[:,accelerometerChannel[ch]]**2))
        features.append(rms)
        featureNames.append('RMS' + ch)
        
        ampRange = np.max(windowData[:,accelerometerChannel[ch]]) - np.min(windowData[:,accelerometerChannel[ch]])
        features.append(ampRange)
        featureNames.append('AmpRange' + ch)
    
    cCMax=[]
    cCLocs=[]
    for i, ch1 in enumerate(accelerometerChannel.keys()):
        for j,ch2 in enumerate(list(accelerometerChannel.keys())[i+1:]):
            crossCorr = np.correlate(windowData[:,accelerometerChannel[ch1]],windowData[:,accelerometerChannel[ch2]],'same')
            crossCorr = crossCorr/(np.std(windowData[:,accelerometerChannel[ch1]]) * np.std(windowData[:,accelerometerChannel[ch2]]))
            
            cCMax.append(np.max(crossCorr))
            cCLocs.append(np.argmax(crossCorr))
    features.append(np.max(cCMax))
    featureNames.append('MaxCC')
    features.append(cCLocs[np.argmax(cCMax)])
    featureNames.append('MaxCCLoc')
        
        #peaks=find_peaks_cwt(windowData[accelerometerChannel[ch],:],np.arange(1,10))
        #peaks=[1,2,3]
        #features.append(len(peaks))
        #featureNames.append('#Movements' + ch)
        
    #features.append(np.max(windowData[0:3,:]))
    #featureNames.append('MaxMovement')
    return featureNames, features